<a href="https://colab.research.google.com/github/JackGraymer/Advanced-GenAI/blob/main/1_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Generative Artificial Intelligence
**Project - Designing a RAG-Based Q&A System for News Retrieval**

**Authors:** Vsevolod Mironov, Pascal Küng, Alvaro Cervan (Group 5)


# Step 1 - Data preparation

**Contribution:** Vsevolod Mironov, Pascal Küng, Alvaro Cervan

**Goal of this step:** To create a clean and well-structured multilingual dataset of news articles with enriched metadata, optimized for efficient indexing and retrieval in a future RAG-based system.

# Loading, Parsing, and Cleaning HTML Files (5 Points)

## 1. Setup of the environment

Below the necessary libraries are installed and loaded into the environment.

In [ ]:
!nvidia-smi

Thu May 15 16:14:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip list

Package                               Version
------------------------------------- ------------------
absl-py                               1.4.0
accelerate                            1.6.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.11.15
aiosignal                             1.3.2
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.6
ale-py                                0.11.0
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.9.0
argon2-cffi                           23.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.2
arviz                                 0.21.0
astropy                               7.0.1
astropy-iers-data                     0.2025.5.5.0.38.14
astunparse                            1.6

In [ ]:
!pip install -q docling==2.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 40.0 MB/s eta 0:00:00
   ━

In [ ]:
from bs4 import BeautifulSoup, Comment
import docling
from docling.document_converter import DocumentConverter, InputFormat, HTMLFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import AcceleratorDevice, AcceleratorOptions
from docling.datamodel.pipeline_options import PipelineOptions

In [ ]:
!pip install -q beautifulsoup4==4.13.4
!pip install -q docling==2.31.0
from bs4 import BeautifulSoup, Comment
import docling
from docling.document_converter import DocumentConverter, InputFormat, HTMLFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import AcceleratorDevice, AcceleratorOptions
from docling.datamodel.pipeline_options import PipelineOptions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 44.0 MB/s eta 0:00:00
   ━

In [ ]:
import os
import re
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import tempfile

In [ ]:
# Set the seed for consistent results
seed_value = 2138247234
random.seed(seed_value)
np.random.seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

Below we mount a shared Google Drive folder as a data storage and define the base path of the folder that will be used in the runtime.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_folder = '/content/drive/MyDrive/AdvGenAI'

## 2. Loading the raw data

### Loading

We go through the subdirectories inside the data-folder. Inside those folders the individual html-files will be read and the content will be saved together with the information of the file-name and the path of the file (to store in which subfolder it was located).

In [ ]:
# Definition of data folder
data_folder = os.path.join(base_folder, 'data')

In [ ]:
%%time
# List to hold the dictionaries
data = []

# Walk through all directories and subdirectories
for root, dirs, files in os.walk(data_folder):
    for file in files:
        if file.endswith('.html'):
            file_path = os.path.join(root, file)

            # Read the content of the HTML file
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            # Add a dictionary to the list
            data.append({
                'folder_path': root,
                'file_name': file,
                'full_path': file_path,
                'html_content': content
            })

# Convert to DataFrame
df = pd.DataFrame(data)

# Optionally save DataFrame, e.g. to CSV or pickle for later use
# df.to_csv('html_files_content.csv', index=False)
# df.to_pickle('html_files_content.pkl')

# Show first rows to verify
print(df.head())

                                         folder_path  \
0  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   
1  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   
2  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   
3  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   
4  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   

                                  file_name  \
0             blog-knutti-klimamodelle.html   
1  scientifica-teaser-kerosin-aus-luft.html   
2            kraftwerk-vor-dem-fenster.html   
3                 eth-plus-initiativen.html   
4   kuenstliche-intelligenz-im-podcast.html   

                                           full_path  \
0  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   
1  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   
2  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   
3  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   
4  /content/drive/MyDrive/AdvGenAI/data/de_news_e...   

                                        htm

In [ ]:
pd.set_option('display.max_colwidth', 50)
df.head()

,folder_path,file_name,full_path,html_content
0,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,blog-knutti-klimamodelle.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,"<div class=""text-image cq-dd-image"">\n<figure ..."
1,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,scientifica-teaser-kerosin-aus-luft.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,"<div class=""text-image cq-dd-image"">\n<p>Viele..."
2,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,kraftwerk-vor-dem-fenster.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,"<div class=""text-image cq-dd-image"">\n<p>Gebäu..."
3,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,eth-plus-initiativen.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,"<div class=""text-image cq-dd-image"">\n<p>Das I..."
4,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,kuenstliche-intelligenz-im-podcast.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,"<div class=""text-image cq-dd-image"">\n<p>Künst..."


In [ ]:
print(df.iloc[10:100, 2].values)

['/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/eth-zuerich-staerkt-forschung-in-den-datenwissenschaften.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/mas-applied-technology-de.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/eth-am-wef-2019.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/doktorat-auf-neue-stufe-heben.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/weniger-fehlalarme-auf-der-intensivstation.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/perlmutt-imitat.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/abschluss-der-disziplinaruntersuchung.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/ploetzliche-stratosphaeren-erwaermung.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/interview-joel-mesot.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2019/01/blog-patt-lilliestam-carbon-tax.html'
 '/

In [ ]:
print(df.iloc[-100:-1, 2].values)

['/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/twelve-professors-appointed.html'
 '/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/speculative-calculations-open-a-backdoor-to-information-theft.html'
 '/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/attosecond-measurement-on-electrons-in-water-clusters.html'
 '/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/heatwaves-thawing-arctic-permafrost.html'
 '/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/dont-attack-the-energy-strategy-with-simplified-assumptions.html'
 '/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/resistance-to-mosaic-disease-explained.html'
 '/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/james-webb-space-telescope-first-image.html'
 '/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/imaging-the-brain-with-ultrasound-waves.html'
 '/content/drive/MyDrive/AdvGenAI/data/en_news_events/2022/07/podcast-series-energy-fact-check.htm

### Checking completeness of loading

**Number of files**

Below we compare the number of documents collected by the function into the Dataframe with a selection of all files in the data folder.

In the check 3 files were discovered that were not part of the dataframe. After inspection it was discovered that those are `.DS_Store`file, for which it makes sense that they were not included.

In [ ]:
# Dataframe
print(f"Number of files in the DataFrame: {len(df)}")

Number of files in the DataFrame: 4390


In [ ]:
# Files in Data folder
print(f"Number of files in the data folder:")
!find "$data_folder" -type f | wc -l

Number of files in the data folder:
4393


In [ ]:
!find "$data_folder" -type f | sort > folder_files.txt
df['full_path'].sort_values().to_csv('df_files.txt', index=False, header=False)
!sort folder_files.txt -o folder_files.txt
!sort df_files.txt -o df_files.txt
!comm -23 folder_files.txt df_files.txt

/content/drive/MyDrive/AdvGenAI/data/de_internal/2013/.DS_Store
/content/drive/MyDrive/AdvGenAI/data/de_internal/2015/.DS_Store
/content/drive/MyDrive/AdvGenAI/data/de_internal/2024/.DS_Store


**Checking for empty files**

Below we print out the rows of the dataframe with empty contents.

In [ ]:
df[df['html_content'].isna()].head(10)

,folder_path,file_name,full_path,html_content


In [ ]:
print(f"Files with no content: {len(df[df['html_content']==''])}")
df[df['html_content']==""].head(10)

Files with no content: 32


,folder_path,file_name,full_path,html_content
265,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,einstein-quiz.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
900,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,pp_pitch_elgar_fleisch.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
1098,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,annette-oxenius-erhaelt-cloetta-preis.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
1102,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,kleider-virtuell-anprobieren.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
1103,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,neue-daten-sprechen-fuer-magma-auf-dem-mars.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
1133,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,proteinformen-zeigen-parkinson-krankheit-an.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
1136,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,kunst-aus-dem-computer.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
1143,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,elektroflugzeug-e-sling-hebt-ab.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
1149,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,besser-verstehen-wie-die-blut-hirn-schranke-fu...,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,
1150,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,wo-giftstoffe-aus-tabak-die-dna-angreifen.html,/content/drive/MyDrive/AdvGenAI/data/de_news_e...,


In [ ]:
print(df.loc[df['html_content']=="", "full_path"].values)

['/content/drive/MyDrive/AdvGenAI/data/de_news_events/2021/08/einstein-quiz.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2016/08/pp_pitch_elgar_fleisch.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2022/10/annette-oxenius-erhaelt-cloetta-preis.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2022/10/kleider-virtuell-anprobieren.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2022/10/neue-daten-sprechen-fuer-magma-auf-dem-mars.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2022/11/proteinformen-zeigen-parkinson-krankheit-an.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2022/11/kunst-aus-dem-computer.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2022/09/elektroflugzeug-e-sling-hebt-ab.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2023/02/besser-verstehen-wie-die-blut-hirn-schranke-funktioniert.html'
 '/content/drive/MyDrive/AdvGenAI/data/de_news_events/2023/02/wo-giftstoffe-aus-taba

After checking the files in the original data source we concluded that those files were empty files and therefore it was not a problem in the process of the data loading. We therefore exclude those rows from the dataframe.

In [ ]:
len(df)

4390

In [ ]:
df = df[~df['html_content'].isna()].copy()
df = df[~(df['html_content']=="")].copy()

In [ ]:
len(df)

4358

In [ ]:
print(df.iloc[:,3].sort_values().values)

['<div class="text-image cq-dd-image">\n<figure class="text-image__image text-image__image--large">\n<a aria-label="Enlarged view: A fault in a medium-voltage installation triggered a power outage on the Hönggerberg campus on Wednesday. (Photo: ETH Zurich/Facility Services)" class="ilb" data-caption="A fault in a medium-voltage installation triggered a power outage on the Hönggerberg campus on Wednesday." data-size="450x800" href="/staffnet/en/news-and-events/internal-news/archive/2023/01/stromausfall-auf-dem-hoenggerberg/_jcr_content/wide_content/textimage/image.imageformat.lightbox.1623277015.jpg">\n<img alt="Enlarged view: A fault in a medium-voltage installation triggered a power outage on the Hönggerberg campus on Wednesday. (Photo: ETH Zurich/Facility Services)" src="/staffnet/en/news-and-events/internal-news/archive/2023/01/stromausfall-auf-dem-hoenggerberg/_jcr_content/wide_content/textimage/image.imageformat.text50percent.1623277015.jpg"/>\n<span class="detail visually-hidden"

In [ ]:
print(df.shape)  # Output: (rows, columns)
print(df.columns)  # List all column names

(4358, 4)
Index(['folder_path', 'file_name', 'full_path', 'html_content'], dtype='object')


### Saving the data to storage

In [ ]:
# Saving the Dataframe to the Google Drive storage
df.to_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-01-raw-data.csv'), index=False)

## 3. Parsing and cleaning the HTML files

### Loading the data from storage

In [ ]:
# Load csv from Google Drive Storage to Dataframe
df = pd.read_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-01-raw-data.csv'))

In [ ]:
pd.set_option('display.max_colwidth', 300)
df[["html_content"]].head(5)

,html_content
0,"<div class=""text-image cq-dd-image"">\n<p>Während andere in den Sechziger-Jahren Lokführer werden wollten, war für Martin Mächler schon immer klar: er würde Forscher werden. Das änderte sich auch auf dem Gymnasium nicht. Wenn er sich nicht gerade mit Mathematik beschäftigte, tüftelte er an Comput..."
1,"<div class=""text-image cq-dd-image"">\n<p><a class=""eth-link"" href=""http://www.library.ethz.ch/de/Ueber-uns/Aktuell/Web-of-Science-Alles-neu-macht-der-Januar"">Weitere Informationen</a><br/> </p>\n</div>\n<div class=""text-image cq-dd-image"">\n<h2>Staffnet</h2>\n<p>Das <a class=""eth-link"" href=""htt..."
2,"<div class=""text-image cq-dd-image"">\n<h2>Staffnet</h2>\n<p>Das <a class=""eth-link"" href=""https://ethz.ch/services/de.html"">Info-Portal</a> für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Zürich.</p>\n</div>\n<div class=""text-image cq-dd-image"">\n<h2>Newslett..."
3,"<div class=""text-image cq-dd-image"">\n<p><a class=""eth-link"" href=""http://www.library.ethz.ch/de/Dienstleistungen/Schulungen-Tutorials-Fuehrungen/Mendeley-Literaturverwaltung"" title=""Weitere Informationen auf der Website der ETH-Bibliothek"">Weitere Informationen</a><br/> </p>\n</div>\n<div class..."
4,"<div class=""text-image cq-dd-image"">\n<p>Das Studierendenleben an der ETH dreht sich momentan nur um eines: Die anstehenden Prüfungen. Während andere sich in den Bergen beim Skifahren vergnügen, sind die Bibliotheken an der ETH bis spät am Abend zum Brechen voll, im Stundentakt begegnet man auf ..."


### BeautifulSoup

#### Definition of cleaning function

Below a function is defined to clean the stored strings of the html-files using `BeautifulSoup`. It extracts the title and main texts of the documents while removing various elements that are not of interest for the further analysis (for example style and navigation elements).

In [ ]:
def clean_html(html_content):
    from bs4 import BeautifulSoup, Comment
    import re

    soup = BeautifulSoup(html_content, 'html.parser')

    # Title extraction
    title = soup.title.get_text(strip=True) if soup.title else ''

    # Remove unwanted elements
    for el in soup(['script', 'style', 'header', 'footer', 'nav', 'iframe', 'meta', 'link']):
        el.decompose()
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    # Replace <br> with newline
    for br in soup.find_all("<br/>"):
        br.replace_with("\n")

    # Get the content from body if exists
    content = soup.body or soup

    # get_text with separator
    clean_text = content.get_text(separator='\n\n').strip()

    # Post-process: Collapse excessive blank lines
    clean_text = re.sub(r'\n{3,}', '\n\n', clean_text)

    return title, clean_text

#### Application of cleaning function on subset

**Add description (trying it out with a subset), the function uses `\n\n` for separation**

In [ ]:
# create a subset of the dataframe for testing
df_test = df.sample(n=5).copy()

In [ ]:
df_test['title'], df_test['clean_content'] = zip(*df_test['html_content'].apply(clean_html))

In [ ]:
pd.set_option('display.max_colwidth', 150)
df_test[['html_content', 'title', 'clean_content']].head()

,html_content,title,clean_content
4040,"<div class=""text-image cq-dd-image"">\n<p>Should he study history or physics? As a teenager in Athens in the 1970s, Konstantinos Boulouchos was int...",,"Should he study history or physics? As a teenager in Athens in the 1970s, Konstantinos Boulouchos was interested in so many things. But while the ..."
2957,"<div class=""text-image cq-dd-image"">\n<p>The ETH Library is the host and organiser of this year's Fall Seminar of the <a class=""eth-link"" href=""ht...",,The ETH Library is the host and organiser of this year's Fall Seminar of the \n\nexternal page\n\nInternational Association of University Librarie...
3133,"<div class=""text-image cq-dd-image"">\n<p>A deuteron is a very simple atomic nucleus made up of just one proton and one neutron — that is, one each...",,"A deuteron is a very simple atomic nucleus made up of just one proton and one neutron — that is, one each of the two nuclear building blocks. An i..."
1462,"<div class=""text-image cq-dd-image"">\n<p>Gestützt auf die Resultate der <a class=""eth-link"" href=""/de/news-und-veranstaltungen/eth-news/news/2017/...",,"Gestützt auf die Resultate der \n\nAdministrativuntersuchung\n\n, die am 25. Oktober 2017 eingeleitet und im Oktober 2018 abgeschlossen wurde, lei..."
1230,"<div class=""text-image cq-dd-image"">\n<p>Bald ist Schluss. Schluss mit der Professur an der ETH, mit Forschungsprojekten und dem Unterricht. Ernst...",,"Bald ist Schluss. Schluss mit der Professur an der ETH, mit Forschungsprojekten und dem Unterricht. Ernst Hafen hat sich gut mit dem Gedanken ange..."


Below we print out the HTML and the cleaned content for each document for comparison. For the cleaned content the double newlines are replaced by `\n---PARAGRAPH BREAK---\n` for better readability.

In [ ]:
from IPython.display import HTML
for idx, row in df_test.iterrows():
    print("-" * 100)
    print(f"Row {idx}:")
    display(HTML(row['html_content']))
    # Display the cleaned text
    print("\nCleaned Text:\n")
    print(row['clean_content'].replace('\n\n', '\n---PARAGRAPH BREAK---\n'))
    print(100 * "-")
    print("\n")

----------------------------------------------------------------------------------------------------
Row 4040:



Cleaned Text:

Should he study history or physics? As a teenager in Athens in the 1970s, Konstantinos Boulouchos was interested in so many things. But while the humanities seemed a bit of a dead end, he was told a physics degree was only good for becoming a teacher, an idea he didn’t quite like either. “And today I find nothing more important than educating young people,” laughs Boulouchos.
---PARAGRAPH BREAK---
The ETH professor is full of enthusiasm when he talks about his students. As a young professor, he says, he hadn’t yet recognised the importance of open and good communication. Today, he knows that everyone has their own story. Those who are absorbed by their own issues, for example, might not be able to perform as well. It’s important to talk about things, because this is the only way to provide effective support. He goes on to say that his students shaped him and helped him to develop – and he hopes he did the same for them. Judging by his former students’ numerous awards an


Cleaned Text:

The ETH Library is the host and organiser of this year's Fall Seminar of the 
---PARAGRAPH BREAK---
external page
---PARAGRAPH BREAK---
International Association of University Libraries
---PARAGRAPH BREAK---
call_made
---PARAGRAPH BREAK---
 (IATUL). For once, the event is also open to non-members of IATUL! 
---PARAGRAPH BREAK---
Please save the date now: 13–15 December 2022.
---PARAGRAPH BREAK---
Under the title "Breaking new ground: scholarly communication and libraries", speakers from libraries, science and publishing houses will shed light on the following topics over the course of three days:
---PARAGRAPH BREAK---
scholarly communication,
---PARAGRAPH BREAK---
 
---PARAGRAPH BREAK---
metrics (bibliometrics and altmetrics),
---PARAGRAPH BREAK---
 
---PARAGRAPH BREAK---
awareness in science.
---PARAGRAPH BREAK---
 
---PARAGRAPH BREAK---
We are pleased to offer you a perfect background and stage for the IATUL Fall Seminar in the surroundings of ETH Zurich and serve as 


Cleaned Text:

A deuteron is a very simple atomic nucleus made up of just one proton and one neutron — that is, one each of the two nuclear building blocks. An international research collaboration, working at the Paul Scherrer Institute PSI, has measured the deuteron more accurately than ever before. The value they obtained for the radius of the deuteron does not, however, correspond to the measurements of other research groups but instead shows a significantly smaller value.
---PARAGRAPH BREAK---
In spite of this contradiction, there is also an agreement: In 2010 the same research group reported on the measurement of individual protons by means of the same method. Then, as well, the measurement clearly showed that the proton is smaller than had been thought to date. Since then, the research community has referred to this situation as “
---PARAGRAPH BREAK---
external page
---PARAGRAPH BREAK---
the proton radius puzzle
---PARAGRAPH BREAK---
call_made
---PARAGRAPH BREAK---
.”  A  furthe


Cleaned Text:

Gestützt auf die Resultate der 
---PARAGRAPH BREAK---
Administrativuntersuchung
---PARAGRAPH BREAK---
, die am 25. Oktober 2017 eingeleitet und im Oktober 2018 abgeschlossen wurde, leitet der Präsident der ETH Zürich das Kündigungsverfahren ein. Die von einem unabhängigen externen Experten durchgeführte Administrativuntersuchung hat schwerwiegendes pflichtwidriges Verhalten über einen längeren Zeitraum hinweg festgestellt. Der Untersuchungsführer empfiehlt eine Auflösung des Arbeitsverhältnisses. Deshalb wird eine Kommission zur Prüfung der Angemessenheit der Kündigung eingesetzt, wie dies 
---PARAGRAPH BREAK---
externe Seite
---PARAGRAPH BREAK---
Art. 13 Abs. 2
---PARAGRAPH BREAK---
call_made
---PARAGRAPH BREAK---
 der Professorenverordnung vorschreibt.
---PARAGRAPH BREAK---
 
---PARAGRAPH BREAK---
 «Der Untersuchungsbericht belegt, dass es sich um inakzeptables Verhalten handelt, das wir nicht tolerieren», sagt ETH-Präsident Lino Guzzella. Er betont aber gleichzeitig,


Cleaned Text:

Bald ist Schluss. Schluss mit der Professur an der ETH, mit Forschungsprojekten und dem Unterricht. Ernst Hafen hat sich gut mit dem Gedanken angefreundet, dass seine aktive Zeit als Professor abgelaufen ist. «Es gibt nichts Unerledigtes», sagt er, am Tisch in seinem Büro sitzend.
---PARAGRAPH BREAK---
Seine Forschungsgruppe hat er aufgelöst. Nur ein Mitarbeiter ist ihm geblieben, und dieser wird sein wohl letztes Forschungsvorhaben weiterführen, bei dem es um die Gesundheit von Honigbienen geht. Seit fünf Jahren hält Hafen mit seiner Frau Bienen auf dem Dach seiner Garage in Zürich – dabei wurde er mit den Problemen der Bienen konfrontiert, allem voran mit den Varroa-Milben, welche den Völkern zusetzen. Dies habe ihn dazu inspiriert und motiviert, dieses Bienen-Projekt zu lancieren.
---PARAGRAPH BREAK---
Sein angestammtes Forschungsfeld verlässt Hafen dafür nicht. Ziel des Projekts ist mit Hilfe der Taufliege Drosophila, dem biologischen Modellsystem, dem Hafen 45 Jahr

The double newlines between header and text are not optimal. Additionally the bullet point lists are not captured as such but. Improvements would be possible but we decided to try if Docling handles the conversion already better with the predefined settings.

Another thing to note is parts of the texts don't give any useful information, such as the "Subscribe to Newsletter" and the "Staffnet" chapter or endings such as the following one: "externe Seite 10.1002/smj.2221 call_made"

#### Application of cleaning function on full Dataframe

Below we apply the conversion to all documents.

In [ ]:
df['bs_html_title'], df['bs_html_content'] = zip(*df['html_content'].apply(clean_html))

In [ ]:
pd.set_option('display.max_colwidth', 150)
df[['html_content', 'bs_html_title', 'bs_html_content']].head(5)

,html_content,bs_html_title,bs_html_content
0,"<div class=""text-image cq-dd-image"">\n<p>Während andere in den Sechziger-Jahren Lokführer werden wollten, war für Martin Mächler schon immer klar:...",,"Während andere in den Sechziger-Jahren Lokführer werden wollten, war für Martin Mächler schon immer klar: er würde Forscher werden. Das änderte si..."
1,"<div class=""text-image cq-dd-image"">\n<p><a class=""eth-link"" href=""http://www.library.ethz.ch/de/Ueber-uns/Aktuell/Web-of-Science-Alles-neu-macht-...",,Weitere Informationen\n\n \n\nStaffnet\n\nDas \n\nInfo-Portal\n\n für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der...
2,"<div class=""text-image cq-dd-image"">\n<h2>Staffnet</h2>\n<p>Das <a class=""eth-link"" href=""https://ethz.ch/services/de.html"">Info-Portal</a> für Mi...",,Staffnet\n\nDas \n\nInfo-Portal\n\n für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Zürich.\n\nNewsletter abo...
3,"<div class=""text-image cq-dd-image"">\n<p><a class=""eth-link"" href=""http://www.library.ethz.ch/de/Dienstleistungen/Schulungen-Tutorials-Fuehrungen/...",,Weitere Informationen\n\n \n\nStaffnet\n\nDas \n\nInfo-Portal\n\n für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der...
4,"<div class=""text-image cq-dd-image"">\n<p>Das Studierendenleben an der ETH dreht sich momentan nur um eines: Die anstehenden Prüfungen. Während and...",,Das Studierendenleben an der ETH dreht sich momentan nur um eines: Die anstehenden Prüfungen. Während andere sich in den Bergen beim Skifahren ver...


#### Saving the data to storage

In [ ]:
# Saving the Dataframe to the Google Drive storage
df.to_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-02-bs.csv'), index=False)

### Docling

#### Definition of the Converter

In [ ]:
# Create AcceleratorOptions for CUDA
cuda_accelerator_options = AcceleratorOptions(device=AcceleratorDevice.CUDA)
InputFormat.HTML: HTMLFormatOption(pipeline_options=PipelineOptions(accelerator_options=cuda_accelerator_options))

In [ ]:
# Initialize the Docling converter
converter = DocumentConverter()

In [ ]:
# Define function for conversion
def html_file_to_markdown(file_path):
    """Convert an HTML file to markdown using Docling"""
    try:
        # Convert the HTML file directly by path
        result = converter.convert(file_path)
        return result.document.export_to_markdown()
    except Exception as e:
        return f"Error converting file {file_path}: {str(e)}"

#### Application of conversion on subset

In [ ]:
# Apply the conversion function using the 'full_path' column
df_test['markdown_content'] = df_test['full_path'].apply(html_file_to_markdown)

# View the result
df_test[['html_content', 'title', 'markdown_content']].head()

,html_content,title,markdown_content
4040,"<div class=""text-image cq-dd-image"">\n<p>Should he study history or physics? As a teenager in Athens in the 1970s, Konstantinos Boulouchos was int...",,"## What influences thinking\n\nSwitzerland has long since become a second home for the professor, who has held Swiss citizenship since 1997. Where..."
2957,"<div class=""text-image cq-dd-image"">\n<p>The ETH Library is the host and organiser of this year's Fall Seminar of the <a class=""eth-link"" href=""ht...",,"## Always up to date\n\nWould you like to always receive the most important internal information and news from ETH Zurich? Then subscribe to the ""..."
3133,"<div class=""text-image cq-dd-image"">\n<p>A deuteron is a very simple atomic nucleus made up of just one proton and one neutron — that is, one each...",,## New experiment creates excitement\n\nThe new research result is actually more than a doubling of the old mystery of the proton radius: Beyond t...
1462,"<div class=""text-image cq-dd-image"">\n<p>Gestützt auf die Resultate der <a class=""eth-link"" href=""/de/news-und-veranstaltungen/eth-news/news/2017/...",,"## Das Entlassungsverfahren gemäss Professorenverordnung\n\nDer Präsident der ETH setzt vor seinem Antrag an die Wahlbehörde, den ETH-Rat, eine Ko..."
1230,"<div class=""text-image cq-dd-image"">\n<p>Bald ist Schluss. Schluss mit der Professur an der ETH, mit Forschungsprojekten und dem Unterricht. Ernst...",,## Den Vater übertrumpfen\n\nZur Biologie kam Hafen durch seinen Biologielehrer am Gymnasium. «Bei Schweizer Jugend forscht hatte ich nie mitgemac...


In [ ]:
from IPython.display import HTML
for idx, row in df_test.iterrows():
    print("-" * 100)
    print(f"Row {idx}:")
    display(HTML(row['html_content']))
    # Display the cleaned text
    print("\nCleaned Text (Markdown):\n")
    print(row['markdown_content'])
    print(100 * "-")
    print("\n")

----------------------------------------------------------------------------------------------------
Row 4040:



Cleaned Text (Markdown):

## What influences thinking

Switzerland has long since become a second home for the professor, who has held Swiss citizenship since 1997. Where does Boulouchos see the biggest differences between Greece and Switzerland? Naturally, in Switzerland everything is more orderly and structured and hence more reliable, he says. But at the end of the day, we are all Greeks, says the professor with a smile. He means this philosophically, like so much of the conversation: “The Greeks Aristotle, Heraclitus and Democritus influenced all our thinking. Logical methodology, the empirical testing of established opinions and doctrines dominate not only, but of course especially, the world of science.” For him, however, it was vital to supplement the thinking of the ancient Greeks with ideas from Eastern philosophy. What particularly fascinates Boulouchos, who loves Japan, is thinking in complex systems: “In Eastern philosophy, humans see themselves as part of a larger system,


Cleaned Text (Markdown):

## Always up to date

Would you like to always receive the most important internal information and news from ETH Zurich? Then subscribe to the "internal news" newsletter and visit Staffnet, the information portal for ETH employees.
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Row 3133:



Cleaned Text (Markdown):

## New experiment creates excitement

The new research result is actually more than a doubling of the old mystery of the proton radius: Beyond that, it can further the search for the true nature of things. “Naturally it can’t be that the deuteron — any more than the proton — has two different sizes,” says Antognini. Thus the scientific community is searching for explanations that can bring the different values back into harmony.

One possible explanation is that a physical force as yet unknown is at work. For the scientists, that is an exciting scenario; it is, however, highly improbable.

A more obvious explanation would be experimental imprecision. “Actually, the mystery could be solved very easily if we assume a minimal experimental problem with the hydrogen spectroscopy,” Antognini explains. Some of the earlier measurements, of both the proton’s size and the deuteron’s size, were based on this method.

Another method for determining the sizes of the proto


Cleaned Text (Markdown):

## Das Entlassungsverfahren gemäss Professorenverordnung

Der Präsident der ETH setzt vor seinem Antrag an die Wahlbehörde, den ETH-Rat, eine Kommission ein, welche über die Angemessenheit der Kündigung befindet und eine Empfehlung abgibt. Diese Empfehlung legt der ETH-Präsident anschliessend zusammen mit seinem eigenen Antrag dem ETH-Rat zum Entscheid vor. Für die Kommission wird die Schulleitung drei Mitglieder selbst bestimmen und zusätzlich die Konferenz des Lehrkörpers auffordern, drei Mitglieder zu bezeichnen. Drei Mitglieder der Kommission müssen Externe sein.

## Kontakt

ETH Zürich
 Medienstelle
 Telefon: +41 44 632 41 41
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Row 1230:



Cleaned Text (Markdown):

## Den Vater übertrumpfen

Zur Biologie kam Hafen durch seinen Biologielehrer am Gymnasium. «Bei Schweizer Jugend forscht hatte ich nie mitgemacht», schmunzelt er. «Ich hatte aber im Gymnasium einen sehr guten Lehrer, der mich für das Fach begeisterte.» Ein zusätzlicher Ansporn sei aber auch gewesen, dass sein Vater, ein Germanist, Deutschlehrer und Rektor am Gymnasium Münchenstein, in Biologie eher schlecht war und der Sohn eine Möglichkeit sah, seinen «Übervater» darin zu übertrumpfen. Also schrieb er für das Studium der Molekular- und Zellbiologie am Biozentrum der Universität Basel ein.

In den Vorlesungen von Walter Gehring, einem bekannten Schweizer Molekular- und Entwicklungsbiologen, stiess Ernst Hafen erstmals auf die Fliege. Gehring habe beschrieben, wie sich in den dotterreichen Insekteneier die Kerne teilen, bis es mehrere tausend davon gibt. Einige davon wandern dann zum hinteren Ende des Eies, wo sie sich in die künftigen Samen- und Eizellen umw

**There seems to be a problem with the conversion of Docling. The part of the HTML before the first heading is not included in the Markdown.**

#### Application of Conversion on full Dataframe

Below we apply the conversion to all documents.

In [ ]:
df['markdown_content_docling'] = df['full_path'].progress_apply(html_file_to_markdown)

100%|██████████| 4358/4358 [00:51<00:00, 83.86it/s]


In [ ]:
pd.set_option('display.max_colwidth', 150)
df[['html_content', 'markdown_content_docling']].head()

,html_content,markdown_content_docling
0,"<div class=""text-image cq-dd-image"">\n<p>Während andere in den Sechziger-Jahren Lokführer werden wollten, war für Martin Mächler schon immer klar:...",## Etwas Sinnvolles für die Menschen schaffen\n\nMehr als 30 Jahre sind seit Beginn seines Doktorats vergangen. In dieser Zeit hat er es weit gebr...
1,"<div class=""text-image cq-dd-image"">\n<p><a class=""eth-link"" href=""http://www.library.ethz.ch/de/Ueber-uns/Aktuell/Web-of-Science-Alles-neu-macht-...",## Staffnet\n\nDas Info-Portal für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Zürich.\n\n## Newsletter abonn...
2,"<div class=""text-image cq-dd-image"">\n<h2>Staffnet</h2>\n<p>Das <a class=""eth-link"" href=""https://ethz.ch/services/de.html"">Info-Portal</a> für Mi...",## Staffnet\n\nDas Info-Portal für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Zürich.\n\n## Newsletter abonn...
3,"<div class=""text-image cq-dd-image"">\n<p><a class=""eth-link"" href=""http://www.library.ethz.ch/de/Dienstleistungen/Schulungen-Tutorials-Fuehrungen/...",## Staffnet\n\nDas Info-Portal für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Zürich.\n\n## Newsletter abonn...
4,"<div class=""text-image cq-dd-image"">\n<p>Das Studierendenleben an der ETH dreht sich momentan nur um eines: Die anstehenden Prüfungen. Während and...","## Zur Person\n\nJulia Wysling\n\n<!-- image -->\n\nIm November 2013 wählte der Mitgliederrat, das oberste Organ des Studierendenverbands VSETH, J..."


#### Saving the data to storage

In [ ]:
# Saving the Dataframe to the Google Drive storage
df.to_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-03-docling.csv'), index=False)

In [ ]:
# For loading
df = pd.read_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-03-docling.csv'))

### Hybrid Approach

#### Definition of the Function for Conversion

In [ ]:
def html_file_to_markdown_bs_docling(file_path):
    """Convert an HTML file to markdown, handling content before first header separately"""
    try:
        # Read the original HTML file
        with open(file_path, 'r', encoding='utf-8') as f:
            html_content = f.read()

        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the first header
        first_header = soup.find(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

        # If no header is found, just use Docling for the whole document
        if not first_header:
            result = converter.convert(file_path)
            return result.document.export_to_markdown()

        # Get the header text to use as a marker
        header_text = first_header.get_text().strip()
        header_level = int(first_header.name[1])
        header_markdown = '#' * header_level + ' ' + header_text

        # Process the entire document with Docling
        result = converter.convert(file_path)
        full_markdown = result.document.export_to_markdown()

        # Find where our header appears in the markdown
        header_index = full_markdown.find(header_markdown)

        # If the header is not found in the markdown, try finding just the header text
        if header_index == -1:
            header_index = full_markdown.find(header_text)
            if header_index == -1:
                # If we still can't find it, return the full markdown
                return full_markdown

        # Extract introduction paragraphs from HTML
        intro_paragraphs = []
        for paragraph in soup.find_all('p'):
            # Only consider paragraphs that appear before the first header
            if (
                hasattr(paragraph, 'sourceline') and
                hasattr(first_header, 'sourceline') and
                paragraph.sourceline < first_header.sourceline
            ):
                text = paragraph.get_text(strip=True)
                if text:
                    intro_paragraphs.append(text)

        # If there are intro paragraphs, combine them
        if intro_paragraphs:
            intro_markdown = "\n\n".join(intro_paragraphs)

            # Check if the intro is already in the markdown before the header
            markdown_before_header = full_markdown[:header_index].strip()

            # If intro is already included (or partially included), use the full markdown
            if intro_markdown in markdown_before_header or any(p in markdown_before_header for p in intro_paragraphs):
                return full_markdown

            # Otherwise, add the intro before the header section
            return intro_markdown + "\n\n" + full_markdown[header_index:]
        else:
            # No intro paragraphs, return the full markdown
            return full_markdown

    except Exception as e:
        return f"Error converting file {file_path}: {str(e)}"

#### Application of conversion on subset

In [ ]:
# Apply the conversion function using the 'full_path' column
df_test['markdown_content_hybrid'] = df_test['full_path'].apply(html_file_to_markdown_bs_docling)

# View the result
df_test[['html_content', 'full_path', 'markdown_content_hybrid']].head()

,html_content,full_path,markdown_content_hybrid
4040,"<div class=""text-image cq-dd-image"">\n<p>Should he study history or physics? As a teenager in Athens in the 1970s, Konstantinos Boulouchos was int...",/content/drive/MyDrive/AdvGenAI/data/en_news_events/2021/06/were-all-greeks.html,"Should he study history or physics? As a teenager in Athens in the 1970s, Konstantinos Boulouchos was interested in so many things. But while the ..."
2957,"<div class=""text-image cq-dd-image"">\n<p>The ETH Library is the host and organiser of this year's Fall Seminar of the <a class=""eth-link"" href=""ht...",/content/drive/MyDrive/AdvGenAI/data/en_internal/2022/07/save-the-date-iatul-fall-seminar-2022.html,The ETH Library is the host and organiser of this year's Fall Seminar of theexternal pageInternational Association of University Librariescall_mad...
3133,"<div class=""text-image cq-dd-image"">\n<p>A deuteron is a very simple atomic nucleus made up of just one proton and one neutron — that is, one each...",/content/drive/MyDrive/AdvGenAI/data/en_news_events/2016/08/deuteron-smaller-than-thought.html,"A deuteron is a very simple atomic nucleus made up of just one proton and one neutron — that is, one each of the two nuclear building blocks. An i..."
1462,"<div class=""text-image cq-dd-image"">\n<p>Gestützt auf die Resultate der <a class=""eth-link"" href=""/de/news-und-veranstaltungen/eth-news/news/2017/...",/content/drive/MyDrive/AdvGenAI/data/de_news_events/2018/10/entlassungsverfahren-eingeleitet.html,"Gestützt auf die Resultate derAdministrativuntersuchung, die am 25. Oktober 2017 eingeleitet und im Oktober 2018 abgeschlossen wurde, leitet der P..."
1230,"<div class=""text-image cq-dd-image"">\n<p>Bald ist Schluss. Schluss mit der Professur an der ETH, mit Forschungsprojekten und dem Unterricht. Ernst...",/content/drive/MyDrive/AdvGenAI/data/de_news_events/2021/07/fliegen-daten-und-sieben-velos.html,"Bald ist Schluss. Schluss mit der Professur an der ETH, mit Forschungsprojekten und dem Unterricht. Ernst Hafen hat sich gut mit dem Gedanken ange..."


In [ ]:
from IPython.display import HTML
for idx, row in df_test.iterrows():
    print("-" * 100)
    print(f"Row {idx}:")
    display(HTML(row['html_content']))
    # Display the cleaned text
    print("\nCleaned Text (Markdown Hybrid):\n")
    print(row['markdown_content_hybrid'])
    print(100 * "-")
    print("\n")

----------------------------------------------------------------------------------------------------
Row 4040:



Cleaned Text (Markdown Hybrid):

Should he study history or physics? As a teenager in Athens in the 1970s, Konstantinos Boulouchos was interested in so many things. But while the humanities seemed a bit of a dead end, he was told a physics degree was only good for becoming a teacher, an idea he didn’t quite like either. “And today I find nothing more important than educating young people,” laughs Boulouchos.

The ETH professor is full of enthusiasm when he talks about his students. As a young professor, he says, he hadn’t yet recognised the importance of open and good communication. Today, he knows that everyone has their own story. Those who are absorbed by their own issues, for example, might not be able to perform as well. It’s important to talk about things, because this is the only way to provide effective support. He goes on to say that his students shaped him and helped him to develop – and he hopes he did the same for them. Judging by his former students’ numerous awards and c


Cleaned Text (Markdown Hybrid):

The ETH Library is the host and organiser of this year's Fall Seminar of theexternal pageInternational Association of University Librariescall_made(IATUL). For once, the event is also open to non-members of IATUL!Please save the date now: 13–15 December 2022.

Under the title "Breaking new ground: scholarly communication and libraries", speakers from libraries, science and publishing houses will shed light on the following topics over the course of three days:

We are pleased to offer you a perfect background and stage for the IATUL Fall Seminar in the surroundings of ETH Zurich and serve as an inspiring site for all participants.

More details about the IATUL Fall Seminar will be made available on ourwebsitein September 2022. We are looking forward to welcoming you to this inspiring event!

Would you like to become a sponsoring partner for this year’s IATUL Fall Seminar? Please contact us via email at.

Moving forward together –Through the power of ne


Cleaned Text (Markdown Hybrid):

A deuteron is a very simple atomic nucleus made up of just one proton and one neutron — that is, one each of the two nuclear building blocks. An international research collaboration, working at the Paul Scherrer Institute PSI, has measured the deuteron more accurately than ever before. The value they obtained for the radius of the deuteron does not, however, correspond to the measurements of other research groups but instead shows a significantly smaller value.

In spite of this contradiction, there is also an agreement: In 2010 the same research group reported on the measurement of individual protons by means of the same method. Then, as well, the measurement clearly showed that the proton is smaller than had been thought to date. Since then, the research community has referred to this situation as “external pagethe proton radius puzzlecall_made.”  A  further analysis of proton data from PSI confirmed the same small value in 2013.

So now it’s the deu


Cleaned Text (Markdown Hybrid):

Gestützt auf die Resultate derAdministrativuntersuchung, die am 25. Oktober 2017 eingeleitet und im Oktober 2018 abgeschlossen wurde, leitet der Präsident der ETH Zürich das Kündigungsverfahren ein. Die von einem unabhängigen externen Experten durchgeführte Administrativuntersuchung hat schwerwiegendes pflichtwidriges Verhalten über einen längeren Zeitraum hinweg festgestellt. Der Untersuchungsführer empfiehlt eine Auflösung des Arbeitsverhältnisses. Deshalb wird eine Kommission zur Prüfung der Angemessenheit der Kündigung eingesetzt, wie diesexterne SeiteArt. 13 Abs. 2call_madeder Professorenverordnung vorschreibt.«Der Untersuchungsbericht belegt, dass es sich um inakzeptables Verhalten handelt, das wir nicht tolerieren», sagt ETH-Präsident Lino Guzzella. Er betont aber gleichzeitig, dass Verfehlungen die Ausnahme und nicht die Regel sind: «Praktisch jede und jeder der über 500 Professorinnen und Professoren der ETH Zürich leisten Tag für Tag hervorra


Cleaned Text (Markdown Hybrid):

Bald ist Schluss. Schluss mit der Professur an der ETH, mit Forschungsprojekten und dem Unterricht. Ernst Hafen hat sich gut mit dem Gedanken angefreundet, dass seine aktive Zeit als Professor abgelaufen ist. «Es gibt nichts Unerledigtes», sagt er, am Tisch in seinem Büro sitzend.

Seine Forschungsgruppe hat er aufgelöst. Nur ein Mitarbeiter ist ihm geblieben, und dieser wird sein wohl letztes Forschungsvorhaben weiterführen, bei dem es um die Gesundheit von Honigbienen geht. Seit fünf Jahren hält Hafen mit seiner Frau Bienen auf dem Dach seiner Garage in Zürich – dabei wurde er mit den Problemen der Bienen konfrontiert, allem voran mit den Varroa-Milben, welche den Völkern zusetzen. Dies habe ihn dazu inspiriert und motiviert, dieses Bienen-Projekt zu lancieren.

Sein angestammtes Forschungsfeld verlässt Hafen dafür nicht. Ziel des Projekts ist mit Hilfe der Taufliege Drosophila, dem biologischen Modellsystem, dem Hafen 45 Jahre seiner Forschungstätig

#### Application of Conversion on full Dataframe

Below we apply the conversion to all documents.

In [ ]:
df['markdown_content_hybrid'] = df['full_path'].progress_apply(html_file_to_markdown_bs_docling)

100%|██████████| 4358/4358 [01:04<00:00, 67.46it/s]


In [ ]:
pd.set_option('display.max_colwidth', 150)
df[['html_content', 'markdown_content_hybrid']].head()

,html_content,markdown_content_hybrid
0,"<div class=""text-image cq-dd-image"">\n<p>Während andere in den Sechziger-Jahren Lokführer werden wollten, war für Martin Mächler schon immer klar:...","Während andere in den Sechziger-Jahren Lokführer werden wollten, war für Martin Mächler schon immer klar: er würde Forscher werden. Das änderte si..."
1,"<div class=""text-image cq-dd-image"">\n<p><a class=""eth-link"" href=""http://www.library.ethz.ch/de/Ueber-uns/Aktuell/Web-of-Science-Alles-neu-macht-...",Weitere Informationen\n\n## Staffnet\n\nDas Info-Portal für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Züric...
2,"<div class=""text-image cq-dd-image"">\n<h2>Staffnet</h2>\n<p>Das <a class=""eth-link"" href=""https://ethz.ch/services/de.html"">Info-Portal</a> für Mi...",## Staffnet\n\nDas Info-Portal für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Zürich.\n\n## Newsletter abonn...
3,"<div class=""text-image cq-dd-image"">\n<p><a class=""eth-link"" href=""http://www.library.ethz.ch/de/Dienstleistungen/Schulungen-Tutorials-Fuehrungen/...",Weitere Informationen\n\n## Staffnet\n\nDas Info-Portal für Mitarbeitende mit den wichtigsten Informationen rund um das Geschehen an der ETH Züric...
4,"<div class=""text-image cq-dd-image"">\n<p>Das Studierendenleben an der ETH dreht sich momentan nur um eines: Die anstehenden Prüfungen. Während and...",Das Studierendenleben an der ETH dreht sich momentan nur um eines: Die anstehenden Prüfungen. Während andere sich in den Bergen beim Skifahren ver...


#### Saving the data to storage

In [ ]:
# Saving the Dataframe to the Google Drive storage
df.to_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-04-hybrid.csv'), index=False)

# Multilingual Text Preprocessing and Cleaning (5 Points)

In [ ]:
# For loading
df = pd.read_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-04-hybrid.csv'))

## Preprocessing

Perform necessary text preprocessing (e.g., removing extra spaces and redundant line breaks, normalizing
Unicode characters, standardizing date formats from different sources), and handle German-specific text
processing (e.g., compound words, umlaut normalization if needed)

In [ ]:
df["markdown_content_hybrid"].head()

,markdown_content_hybrid
0,Während andere in den Sechziger-Jahren Lokführ...
1,Weitere Informationen\n\n## Staffnet\n\nDas In...
2,## Staffnet\n\nDas Info-Portal für Mitarbeiten...
3,Weitere Informationen\n\n## Staffnet\n\nDas In...
4,Das Studierendenleben an der ETH dreht sich mo...


## Metadata

Store the cleaned text and its metadata in a structured format suitable for retrieval (e.g., JSON, CSV, or a
database) with fields such as **language, title, date, source**

The following step enriches the dataframe by adding the date (month and year) extracted from the file path, as they are organized by date.

In [ ]:
# Function to extract year and month from the folder path
def extract_year_month(path):

    if isinstance(path, str):  # Check if path is a string.
        parts = path.split('/')
        if len(parts) >= 2: # Check if the path has at least two parts
            month = parts[-1]
            year = parts[-2]
            return year, month
        else:
             return None, None
    else:
        return None, None #Handles the case where the input is not a string

# Apply the function to create new columns 'year' and 'month'
df[['year', 'month']] = df['folder_path'].apply(lambda x: pd.Series(extract_year_month(x)))


Extracts the type of document title and the language from the folder path structure.

In [ ]:
def extract_language_type(path):
    if isinstance(path, str):
        parts = path.split('/')
        if len(parts) >= 4:  # Check for the third element from the end
            third_from_end = parts[-3]
            lang_type_parts = third_from_end.split('_')
            language = lang_type_parts[0] if lang_type_parts[0] in ('de', 'en') else None
            Type = 'internal' if len(lang_type_parts) > 1 and lang_type_parts[1] == 'internal' else \
                   'news events' if len(lang_type_parts) > 1 and lang_type_parts[1] == 'news' else None
            return language, Type
        else:
            return None, None
    else:
        return None, None

# Apply the function to create new columns 'language' and 'Type'
df[['language', 'type']] = df['folder_path'].apply(lambda x: pd.Series(extract_language_type(x)))

Extracts HTML file name and adds it to the dataframe. This could be useful as a backup title, as many files do not contain a `<meta>` tag with the title or a `<h1>` tag for the title.

In [ ]:
# Function to extract and format the title from the file_name
def extract_and_format_title(file_name):
    """
    Extracts the title from the filename, removes the '.html' extension,
    replaces hyphens with spaces, and capitalizes only the first letter of the first word.

    Args:
        file_name (str): The filename.

    Returns:
        str: The formatted title, or None if the input is not a string.
    """
    if isinstance(file_name, str):
        title = file_name.replace(".html", "").replace("-", " ")
        words = title.split()
        if words:
            words[0] = words[0].capitalize()
            title = " ".join(words)
        return title
    else:
        return None

# Apply the function to create the 'html_title' column
df['html_title'] = df['file_name'].apply(extract_and_format_title)

In [ ]:
# Print the updated DataFrame
print(df.head())

                                         folder_path  \
0  /content/drive/MyDrive/AdvGenAI/data/de_intern...   
1  /content/drive/MyDrive/AdvGenAI/data/de_intern...   
2  /content/drive/MyDrive/AdvGenAI/data/de_intern...   
3  /content/drive/MyDrive/AdvGenAI/data/de_intern...   
4  /content/drive/MyDrive/AdvGenAI/data/de_intern...   

                                          file_name  \
0                               der-r-pionier1.html   
1  web-of-science-alles-neu-macht-der--januar-.html   
2    swiss-life-sciences-2014-experten-gesucht.html   
3                 mendeley-literaturverwaltung.html   
4                            mehr-orientierung.html   

                                           full_path  \
0  /content/drive/MyDrive/AdvGenAI/data/de_intern...   
1  /content/drive/MyDrive/AdvGenAI/data/de_intern...   
2  /content/drive/MyDrive/AdvGenAI/data/de_intern...   
3  /content/drive/MyDrive/AdvGenAI/data/de_intern...   
4  /content/drive/MyDrive/AdvGenAI/data/de_intern...

In [ ]:
print(df.columns)
# drop 'bs_html_content'and 'markdown_content_docling' columns
df1 = df.drop(columns=['bs_html_content', 'markdown_content_docling'])
print(df1.columns)

Index(['folder_path', 'file_name', 'full_path', 'html_content',
       'bs_html_title', 'bs_html_content', 'markdown_content_docling',
       'markdown_content_hybrid', 'year', 'month', 'language', 'type',
       'html_title'],
      dtype='object')
Index(['folder_path', 'file_name', 'full_path', 'html_content',
       'bs_html_title', 'markdown_content_hybrid', 'year', 'month', 'language',
       'type', 'html_title'],
      dtype='object')


In [ ]:
# save file as csv
df.to_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-05-metadata.csv'), index=False)

## Metadata Extraction from Content (NLP)

In the following steps, information about the text will be extracted and added as metadata to the dataframe, fields such as **main content, named entities, topics, keywords, summary**

In [ ]:
# load step 5 dataset
pd.read_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-05-metadata.csv'))

Performs necessary text preprocessing (e.g., removing extra spaces and redundant line breaks, normalizing
Unicode characters, and handle German-specific text
processing 'ß'.

We decided that there is no need to process and unify date structure as the date is already stracted from the path of the files. We might lose some information regarding the day if included in some texts, but with a span of over a decade, that level of granularity wont be necessary.

Regarding the German specific processing such as umlaut or compound words:
- After normalizing unicode characters, we can see that umlauts and other characters are not an issue and are displayed proyerly, so there is no need for further processing.
- Compound words are an essential part of the German language, and processing it could affect negatively the meaning and understandability of the text explained, as explained by our german speaking colleague Pascal, hence we decided against of processing them.

In [ ]:


#print an example of the text in the markdown_content_hybrid column
print(df['markdown_content_hybrid'].iloc[0])

def preprocess_text(text):
	# Remove extra spaces and redundant line breaks
	text = re.sub(r'\s+', ' ', text)
	text = text.strip()

	# Normalize Unicode characters (if needed)
	text = text.encode('utf-8').decode('utf-8')

	# Handle German-specific text processing (e.g., compound words, umlaut normalization)
	text = text.replace('ß', 'ss')

	return text

# Apply the preprocessing function to the 'markdown_content_hybrid' column
df['content'] = df['markdown_content_hybrid'].progress_apply(preprocess_text)

print(df['content'].iloc[0])

The next step processes the (`content` column) and its language (`language` column)to perform the following tasks:

1. **Loads NLP Models**: Initializes spaCy models for English and German text processing.
2. **Extracts Features**: Defines a function `extract_all` to extract:
   - Named entities using spaCy.
   - Keywords using KeyBERT.
   - Topics using Gensim's LDA.
   - A summary (heuristic: two longest sentences).
3. **Applies the Function**: Processes each row of the DataFrame using `progress_apply` and adds the extracted features as new columns (`named_entities`, `topics`, `keywords`, `summary`).
4. **Displays Results**: Prints the first few rows of the updated DataFrame.

In [ ]:
# --- Install required packages if needed ---
# pip install spacy keybert gensim tqdm
# python -m spacy download en_core_web_sm
# python -m spacy download de_core_news_sm

import pandas as pd
import spacy
from keybert import KeyBERT
from gensim import corpora, models
from tqdm import tqdm

# Load spaCy models
nlp_en = spacy.load("en_core_web_sm")
nlp_de = spacy.load("de_core_news_sm")

# Init KeyBERT
kw_model = KeyBERT()

# Enable progress_apply
tqdm.pandas()

# --- Define the extraction function ---
def extract_all(text, lang):
    # Select language-specific spaCy model
    if lang == "de":
        nlp = nlp_de
    else:
        nlp = nlp_en

    doc = nlp(text)

    # Named Entities
    entities = list(set((ent.text, ent.label_) for ent in doc.ents))

    # Keywords
    keywords = kw_model.extract_keywords(text, top_n=5)
    keywords = [kw[0] for kw in keywords]

    # Topics using LDA (via Gensim)
    tokens = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop]
    dictionary = corpora.Dictionary([tokens])
    corpus = [dictionary.doc2bow(tokens)]
    try:
        lda_model = models.LdaModel(corpus, num_topics=1, id2word=dictionary, passes=4)
        topics = [word for word, _ in lda_model.show_topic(0)]
    except:
        topics = []

    # Summary (heuristic: longest 2 sentences)
    sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 20]
    sentences = sorted(sentences, key=lambda s: len(s), reverse=True)
    summary = " ".join(sentences[:2]) if sentences else ""

    return pd.Series({
        "named_entities": entities,
        "topics": topics,
        "keywords": keywords,
        "summary": summary
    })

# --- Apply it to your DataFrame ---
# Make sure your df has 'content' and 'language' columns
df = df.dropna(subset=["content", "language"])
df[["named_entities", "topics", "keywords", "summary"]] = df.progress_apply(
    lambda row: extract_all(row["content"], row["language"]),
    axis=1
)

# Optional: display result
print(df[["content", "language", "named_entities", "topics", "keywords", "summary"]].head())


In [ ]:
# save the updated dataframe to a new CSV file
df.to_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-06-NLP-processed.csv'), index=False)

In [ ]:
# load dataset
df = pd.read_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-06-NLP-processed.csv'))

### Evaluation of Summary

This code uses `sumy` library to extract the summary from the `content` using Latent Semantic Analysis (LSA). Tt processes the text content, applies the `LsaSummarizer` to extract key sentences based on topic relevance, and removes stop words to enhance readability. The summarization function is then applied to each row in the DataFrame, creating a new column with summarized text

In [ ]:
import pandas as pd
from sumy.parsers.plaintext import PlaintextParser  # Corrected import
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.utils import get_stop_words

import nltk
nltk.download('punkt_tab')

# Function to summarize text based on language
i=0
def summarize_text(row):
    global i
    language = row["language"]
    parser = PlaintextParser.from_string(row["content"], Tokenizer(language))  # Corrected usage
    summarizer = LsaSummarizer()
    summarizer.stop_words = get_stop_words(language)

    summary_sentences = summarizer(parser.document, 2)  # Get top 2 sentences
    i=i+1
    print(i)
    return " ".join([str(sentence) for sentence in summary_sentences])

# Apply summarization
df["summary_summy"] = df.apply(summarize_text, axis=1)

print(df)


In [ ]:
df.to_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-07-summaryv2.csv'), index=False)

In [ ]:
# load dataset
df = pd.read_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-07-summaryv2.csv'))
print(df.columns)

Index(['folder_path', 'file_name', 'full_path', 'html_content',
       'bs_html_title', 'markdown_content_hybrid', 'year', 'month', 'language',
       'type', 'html_title', 'content', 'named_entities', 'topics', 'keywords',
       'summary', 'summary_summy'],
      dtype='object')


In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import pandas as pd
from bert_score import score

# Make sure content and summaries are strings
df = df.dropna(subset=['content', 'summary', 'summary_summy'])
contents = df['content'].astype(str).tolist()
summaries_1 = df['summary'].astype(str).tolist()
summaries_2 = df['summary_summy'].astype(str).tolist()

# Compute BERTScore (Precision, Recall, F1) for both sets of summaries
P1, R1, F1 = score(summaries_1, contents, lang="en", verbose=True)
P2, R2, F2 = score(summaries_2, contents, lang="en", verbose=True)

# Add results to DataFrame
df['bertscore_summary_f1'] = F1.tolist()
df['bertscore_summy_f1'] = F2.tolist()

# Compare average F1 scores
mean_f1_summary = F1.mean().item()
mean_f1_summy = F2.mean().item()

print(f"Mean BERTScore F1 - summary:      {mean_f1_summary:.4f}")
print(f"Mean BERTScore F1 - summary_summy: {mean_f1_summy:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/134 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/69 [00:00<?, ?it/s]

done in 221.03 seconds, 19.71 sentences/sec
Mean BERTScore F1 - summary:      0.8648
Mean BERTScore F1 - summary_summy: 0.8640


The result of comparing both summaries ends in a tie:
- Mean BERTScore F1 - summary:      0.8648
- Mean BERTScore F1 - summary_summy: 0.8640

In [ ]:
# filter the dataframe by 'language' = 'en' and take 10 random samples with a fixed seed
sample = df[df['language'] == 'en'].sample(10, random_state=42)[['summary', 'summary_summy']]

# print the samples in pairs
for i, row in sample.iterrows():
	print(f"Summary: {row['summary']}")
	print(f"Summy: {row['summary_summy']}")
	print()

Summary: The hardware developed by xorlab, a mini-server weighing 31 kilos, stands before the company’s actual server infrastructure as a front line of defence against potential hacker attacks via emails, PDFs, Word or Excel documents, or websites in general. A study by Frost &amp; Sullivan estimated that the market for IT security network applications, to which xorlab’s system also belongs, was worth more than 500 million US dollars in 2014, and is set to grow to around 3.5 billion US dollars by 2019.
Summy: The hardware developed by xorlab, a mini-server weighing 31 kilos, stands before the company’s actual server infrastructure as a front line of defence against potential hacker attacks via emails, PDFs, Word or Excel documents, or websites in general. ## On the verge of the first critical trials xorlab’s innovative product might soon be in great demand: in recent years there has been a massive increase in the number of cyber attacks on companies and governments.

Summary: … but whe

The heuristic-based approach selects the longest sentences, assuming they are the most relevant, but often fails to capture the true essence of the content, producing long chunks without contextual analysis.

In contrast, summy uses advanced algorithms to extract meaningful sentences, creating concise and focused summaries based on the data. When summaries differ, summy consistently provides more context-aware and coherent outputs by inferring importance rather than relying on sentence length.

While both methods may achieve similar BERT scores, summy demonstrates a deeper understanding of the content, producing summaries that are both accurate and contextually meaningful.

Hence we will keep only the sumy summary and delete the helper columns on the dataframe

In [ ]:
# Load again the dataset to avoid deleting several columns created in the previous steps
df = pd.read_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-07-summaryv2.csv'))

# Delete 'summary' column
df.drop(columns=['summary'], inplace=True)

# rename column 'summary_summy' to summary
df.rename(columns={'summary_summy': 'summary'}, inplace=True)
print(df.columns)



Index(['folder_path', 'file_name', 'full_path', 'html_content',
       'bs_html_title', 'markdown_content_hybrid', 'year', 'month', 'language',
       'type', 'html_title', 'content', 'named_entities', 'topics', 'keywords',
       'summary'],
      dtype='object')


In [ ]:
df.to_csv(os.path.join(base_folder, 'Stage1/Working-dir/Stage1-08-final.csv'), index=False)